In [204]:
import numpy as np
#import random
import codecs
import math
import pandas
from scipy.spatial import Voronoi#, voronoi_plot_2d
from data import DATA
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi

In [205]:
origen_varientes = "../media/supermatrix.csv"
origen_HSIC= "../media/hsic_samp500.csv"

variantes = pandas.read_csv(origen_varientes, index_col=[0,1], sep="\t")
hsic = pandas.read_csv(origen_HSIC, index_col=[0,1], sep="\t")
hsic.loc['Latitud']=0
hsic.loc['Longitud']=0

hsic_valid = hsic.loc[(hsic['HSIC'] > 0.009)]
hsic_valid = hsic_valid.loc[(hsic_valid['PValue'] < 0.05)]



In [206]:
valid_variants = variantes.ix[list(hsic_valid.index.values)]
variantes.index.values

array([('Latitud', '####XXX####'), ('Longitud', '####XXX####'),
       ('pizarra', 'T3L193M184I_TABLERO'), ...,
       ('vieja', 'T3L007M007FAM_ESPOSA_II'),
       ('víbora', 'T3L007M007FAM_ESPOSA_II'),
       ('ñervito', 'T3L007M007FAM_ESPOSA_II')], dtype=object)

In [207]:
coordinates = variantes.iloc[0:2,:].transpose()
points = np.array(coordinates)
vor = Voronoi(points)

coordinates.columns=['Latitud', 'Longitud']

In [208]:
plt.figure(figsize=(80,40))

# setup Lambert Conformal basemap.
m = Basemap(width=1500000,height=2000000,projection='lcc',
            resolution='h',lat_1=15.,lat_2=25,lat_0=3.9,lon_0=-73.)
# draw coastlines.
m.drawcoastlines(linewidth=4)
m.drawcountries(linewidth=4)
m.drawmapboundary(fill_color='aquamarine')
m.fillcontinents(color="ivory",lake_color="aquamarine")
#m.drawstates()
#m.shadedrelief()
#m.etopo()

# draws locations
for index, row in coordinates.iterrows():
    m.scatter(row['Longitud'],row['Latitud'],s=50,marker='o',color='k',latlon=True)
    m.tissot(row['Longitud'],row['Latitud'],radius_deg=10/200,npts=20)
    x,y=m(row['Longitud'],row['Latitud'])
    plt.text(x,y,index,fontsize=7)

In [209]:
def weights(c1,c2):
    absolute_difference=abs(valid_variants.iloc[:,c1]-valid_variants.iloc[:,c2]).sum()
    return absolute_difference

In [210]:
min_weight=99999999999999999999999999
max_weight=0.0000000000000000000000000000000001

for i in range(len(vor.ridge_vertices)):
    vertex1,vertex2=vor.ridge_vertices[i]  # points in the map defining the line
    city1,city2=vor.ridge_points[i]
    line_weight=weights(city1,city2)
    if line_weight>max_weight:
        max_weight=line_weight
    if line_weight<min_weight:
        min_weight=line_weight

print(min_weight,max_weight)

0.0 59.0


In [211]:
for i in range(len(vor.ridge_vertices)):
    vertex1,vertex2=vor.ridge_vertices[i]  # points in the map defining the line
    city1,city2=vor.ridge_points[i]  # indexes of the cities separated by the line   
    #gets the line's weight
    line_weight=(weights(city1,city2)-min_weight)/(max_weight-min_weight)
    if vertex1==-1 or vertex2==-1:
        continue
    latitude1,longitude1=vor.vertices[vertex1]
    latitude2,longitude2=vor.vertices[vertex2]
    x,y=m([longitude1,longitude2],[latitude1,latitude2])    
    m.plot(x,y,color=(line_weight,1-line_weight, 1-line_weight),linewidth=10*(line_weight))

#plt.show()
plt.savefig('../static/Localizaciones_ALECT3_Colombia_Voroni.png', format='png', dpi=200)